In [1]:
!pip install --quiet datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
import torch
import pandas as pd
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
import os
from accelerate import Accelerator
from tqdm import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import datasets

In [3]:
df = pd.read_csv('/content/final_randomized_augmented.csv')

In [4]:
df['label'] = 0
df

,id,text,Anger,Fear,Joy,Sadness,Surprise,label
0,eng_train_track_a_00915,"She never listens, no matter how many times I ...",1.0,1.0,0.0,0.0,0.0,0
1,eng_train_track_a_00416,But one criminal shoot from gun to my hand.,1.0,1.0,0.0,0.0,0.0,0
2,eng_train_track_a_03513,The fear of the unknown kept me frozen in plac...,0.0,0.0,1.0,1.0,0.0,0
3,eng_train_track_a_01414,What demon likes peas?,0.0,0.0,0.0,0.0,1.0,0
4,eng_train_track_a_03305,The last thing I expected was to find myself i...,0.0,1.0,0.0,1.0,0.0,0
...,...,...,...,...,...,...,...,...
4188,eng_train_track_a_01362,He keeps eating my leftovers without asking.,1.0,0.0,0.0,0.0,0.0,0
4189,eng_train_track_a_00558,"Pretty eerie, but awesome",0.0,1.0,1.0,0.0,1.0,0
4190,eng_train_track_a_00912,They parked so close to my car that I couldn’t...,1.0,1.0,0.0,0.0,0.0,0
4191,eng_train_track_a_03355,It was one of those moments where time seemed ...,0.0,1.0,0.0,0.0,1.0,0


In [5]:
type(df.iloc[0]['Anger'])

numpy.float64

In [6]:
df['labels'] = df.apply(lambda row: [
    float(row['Anger']), float(0), float(0),
    float(row['Fear']),
    float(row['Joy']), float(0), float(0), float(0),
    float(row['Sadness']),
    float(row['Surprise']), float(0)
], axis=1)
df = df.drop(columns=['id','Joy','Fear','Anger','Sadness','Surprise'])


In [7]:
df = df.drop(columns=['label'])
df

,text,labels
0,"She never listens, no matter how many times I ...","[1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,But one criminal shoot from gun to my hand.,"[1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,The fear of the unknown kept me frozen in plac...,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, ..."
3,What demon likes peas?,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,The last thing I expected was to find myself i...,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, ..."
...,...,...
4188,He keeps eating my leftovers without asking.,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4189,"Pretty eerie, but awesome","[0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, ..."
4190,They parked so close to my car that I couldn’t...,"[1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4191,It was one of those moments where time seemed ...,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [8]:
from datasets import Dataset

dataset = Dataset.from_pandas(df)  # Use only the desired columns

# Optionally, print the dataset to check
print(dataset)

Dataset({
    features: ['text', 'labels'],
    num_rows: 4193
})


In [9]:
from transformers import AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-large-emotion-latest", device_map="auto")

def preprocess_function(examples):
    # Tokenize the text
    inputs = tokenizer(examples["text"], truncation=True, padding="max_length", max_length=256)

    # Convert labels to float, handling NoneType by treating it as 0, even in nested lists
    inputs["labels"] = [
        torch.tensor([l if l is not None else 0 for l in label], dtype=torch.float)
        if isinstance(label, list) else torch.tensor(label if label is not None else 0, dtype=torch.float)
        for label in examples["labels"]
    ]

    return inputs

encoded_dataset = dataset.map(preprocess_function, batched=True)
encoded_dataset


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

Map:   0%|          | 0/4193 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'labels', 'input_ids', 'attention_mask'],
    num_rows: 4193
})

In [10]:
print(encoded_dataset[0]["labels"])

[1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [11]:
encoded_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

In [12]:
print(encoded_dataset[0]["labels"])  # Should output a tensor of floats: e.g., tensor([1., 0., 0., ..., 0., 1.])


tensor([1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.])


In [13]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-large-emotion-latest", device_map="auto")
model

config.json:   0%|          | 0.00/1.22k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=Tru

In [14]:
dataset = encoded_dataset.train_test_split(test_size=0.2)

In [15]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 3354
    })
    test: Dataset({
        features: ['text', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 839
    })
})

In [16]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    # Sigmoid is typically used for multi-label classification to get probabilities
    probs = torch.sigmoid(torch.tensor(logits)).numpy()
    # Convert probabilities to binary predictions (0 or 1)
    predictions = (probs > 0.5).astype(int)

    # Calculate metrics
    f1 = f1_score(labels, predictions, average='macro', zero_division=1)
    precision = precision_score(labels, predictions, average='macro', zero_division=1)
    recall = recall_score(labels, predictions, average='macro', zero_division=1)
    accuracy = accuracy_score(labels, predictions)  # Multi-label accuracy

    return {
        "accuracy": accuracy,
        "f1": f1,
        "precision": precision,
        "recall": recall
    }

In [17]:
from transformers import TrainingArguments, Trainer
# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    report_to="none"   # Disable wandb logging if not needed
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    compute_metrics=compute_metrics
)

In [18]:
# Train the model
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.118062,0.662694,0.919882,0.943626,0.899100
2,0.142800,0.130589,0.665077,0.919827,0.938950,0.902901


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.118062,0.662694,0.919882,0.943626,0.899100
2,0.142800,0.130589,0.665077,0.919827,0.938950,0.902901
3,0.076200,0.130915,0.676996,0.927352,0.930941,0.925124
4,0.043500,0.137001,0.674613,0.928528,0.934824,0.922783
5,0.022200,0.141545,0.684148,0.930423,0.936724,0.924825


TrainOutput(global_step=2100, training_loss=0.06864348644302005, metrics={'train_runtime': 2914.0907, 'train_samples_per_second': 5.755, 'train_steps_per_second': 0.721, 'total_flos': 7814482105697280.0, 'train_loss': 0.06864348644302005, 'epoch': 5.0})

In [29]:
df = pd.read_csv('/content/eng.csv')

In [30]:
true_labels = []
for row in range(len(df)):
    true_label = [int(df.loc[row]['Joy']),
                  int(df.loc[row]['Fear']),
                  int(df.loc[row]['Anger']),
                  int(df.loc[row]['Sadness']),
                  int(df.loc[row]['Surprise'])]
    true_labels.append(true_label)
print(true_labels)

[[1, 0, 0, 1, 0], [1, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 1, 1, 1, 1], [1, 0, 0, 0, 0], [1, 0, 0, 0, 0], [0, 1, 0, 1, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 0], [0, 1, 0, 0, 1], [1, 0, 0, 0, 0], [0, 0, 0, 0, 0], [1, 0, 0, 0, 0], [1, 0, 0, 0, 0], [1, 0, 0, 0, 0], [0, 1, 0, 1, 0], [0, 1, 0, 0, 1], [1, 1, 0, 0, 1], [0, 1, 0, 1, 0], [1, 0, 0, 0, 0], [0, 1, 0, 0, 1], [1, 0, 1, 0, 1], [0, 1, 0, 1, 0], [0, 1, 0, 1, 1], [0, 1, 0, 0, 1], [1, 0, 0, 0, 0], [1, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1, 0], [0, 1, 0, 0, 0], [0, 1, 0, 1, 1], [0, 1, 0, 1, 0], [0, 1, 0, 1, 0], [0, 1, 0, 1, 0], [0, 1, 0, 0, 0], [0, 1, 1, 1, 0], [0, 1, 0, 1, 1], [0, 1, 0, 0, 0], [0, 1, 0, 1, 0], [1, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 1, 0, 0, 1], [0, 1, 0, 0, 1], [0, 1, 1, 0, 0], [0, 1, 0, 1, 0], [1, 0, 0, 0, 0], [1, 0, 0, 1, 0], [0, 0, 0, 0, 0], [1, 0, 0, 0, 0], [0, 1, 0, 0, 1], [0, 1, 0, 0, 0], [0, 1, 0, 0, 1], [0, 1, 0, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 1, 0, 0, 

In [31]:
emotion_classifier = pipeline("text-classification", model=model, tokenizer=tokenizer, top_k=None)
correct = 0
pred_labels = []
for row in tqdm(range(len(df))):
    joy, fear, anger, sadness, surprise = 0,0,0,0,0


    # Define some sample text for emotion classification
    text = df.loc[row]["text"]
    # print(text)
    # Get emotion predictions
    predictions = emotion_classifier(text)
    # print(predictions)

    # Display the results
    for prediction in predictions[0]:
        # print(prediction['score'])
        # prediction['score'] = 1 if prediction['score']>0.5 else 0
        if prediction['label'] == 'joy' and prediction['score'] > 0.5:
            # print("Inside")
            joy = 1
        # else: joy = 0
        if prediction['label'] == 'fear' and prediction['score'] > 0.5:
            # print("Inside")
            fear = 1
        # else: fear = 0
        if prediction['label'] == 'anger' and prediction['score'] > 0.5:
            # print("Inside")
            anger = 1
        # else: anger = 0
        if prediction['label'] == 'sadness' and prediction['score'] > 0.5:
            # print("Inside")
            sadness = 1
        # else: sadness = 0
        if prediction['label'] == 'surprise' and prediction['score'] > 0.5:
            # print("Inside")
            surprise = 1
        # else: surprise = 0
    # print(joy, fear, anger, sadness, surprise )
    if (df.loc[row]['Joy'] == joy and
        df.loc[row]['Fear'] == fear and
        df.loc[row]['Anger'] == anger and
        df.loc[row]['Sadness'] == sadness and
        df.loc[row]['Surprise'] == surprise):
        correct = correct + 1
    pred_labels.append([joy, fear, anger, sadness, surprise])
print(correct/len(df))

        # print(f"Emotion: {prediction['label']}, Score: {prediction['score']:.4f}")

100%|██████████| 2768/2768 [00:49<00:00, 56.35it/s]

0.7478323699421965


In [32]:
precision_micro = precision_score(true_labels, pred_labels, average='micro')
recall_micro = recall_score(true_labels, pred_labels, average='micro')

# average='macro': Calculate metrics for each label, then average the results equally
precision_macro = precision_score(true_labels, pred_labels, average='macro')
recall_macro = recall_score(true_labels, pred_labels, average='macro')

# average='weighted': Calculate metrics for each label, then average the results weighted by the number of true instances for each label
precision_weighted = precision_score(true_labels, pred_labels, average='weighted')
recall_weighted = recall_score(true_labels, pred_labels, average='weighted')

print(f"Micro Precision: {precision_micro}")
print(f"Micro Recall: {recall_micro}")

print(f"Macro Precision: {precision_macro}")
print(f"Macro Recall: {recall_macro}")

print(f"Weighted Precision: {precision_weighted}")
print(f"Weighted Recall: {recall_weighted}")

Micro Precision: 0.8709394205443372
Micro Recall: 0.9153402537485582
Macro Precision: 0.8611746442828462
Macro Recall: 0.9239597718996467
Weighted Precision: 0.882250768058716
Weighted Recall: 0.9153402537485582


In [33]:
accuracy = accuracy_score(true_labels, pred_labels)
micro_f1 = f1_score(true_labels, pred_labels, average='micro')
macro_f1 = f1_score(true_labels, pred_labels, average='macro')
weighted_f1 = f1_score(true_labels, pred_labels, average='weighted')
print(f"Accuracy: {accuracy*100}")
print(f"Micro F1 Score: {micro_f1}")
print(f"Macro F1 Score: {macro_f1}")
print(f"Weighted F1 Score: {weighted_f1}")

Accuracy: 74.78323699421965
Micro F1 Score: 0.8925880103475424
Macro F1 Score: 0.8884144190998182
Weighted F1 Score: 0.8945339626392536


In [20]:
df = pd.read_csv('/content/eng_a.csv')

In [21]:
# Inferencing on dev set and storing the predicted outputs
emotion_classifier = pipeline("text-classification", model=model, tokenizer=tokenizer, top_k=None)
pred_labels = []
for row in tqdm(range(len(df))):
    joy, fear, anger, sadness, surprise = 0,0,0,0,0


    # Define some sample text for emotion classification
    text = df.loc[row]["text"]
    # print(text)
    # Get emotion predictions
    predictions = emotion_classifier(text)
    # print(predictions)

    # Display the results
    for prediction in predictions[0]:
        # print(prediction['score'])
        # prediction['score'] = 1 if prediction['score']>0.5 else 0
        if prediction['label'] == 'joy' and prediction['score'] > 0.5:
            # print("Inside")
            joy = 1
        # else: joy = 0
        if prediction['label'] == 'fear' and prediction['score'] > 0.5:
            # print("Inside")
            fear = 1
        # else: fear = 0
        if prediction['label'] == 'anger' and prediction['score'] > 0.5:
            # print("Inside")
            anger = 1
        # else: anger = 0
        if prediction['label'] == 'sadness' and prediction['score'] > 0.5:
            # print("Inside")
            sadness = 1
        # else: sadness = 0
        if prediction['label'] == 'surprise' and prediction['score'] > 0.5:
            # print("Inside")
            surprise = 1
        # else: surprise = 0
    # print(joy, fear, anger, sadness, surprise )

    pred_labels.append([joy, fear, anger, sadness, surprise])


        # print(f"Emotion: {prediction['label']}, Score: {prediction['score']:.4f}")

100%|██████████| 116/116 [00:01<00:00, 59.16it/s]


In [22]:
pred_labels

[[0, 1, 0, 0, 1],
 [0, 1, 0, 1, 1],
 [0, 0, 0, 0, 1],
 [0, 1, 0, 1, 0],
 [0, 1, 0, 0, 1],
 [0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0],
 [0, 0, 0, 1, 0],
 [1, 0, 0, 0, 0],
 [0, 1, 0, 1, 0],
 [1, 0, 0, 0, 0],
 [0, 1, 0, 1, 0],
 [0, 0, 1, 1, 1],
 [0, 1, 0, 1, 1],
 [1, 0, 0, 0, 0],
 [1, 0, 0, 0, 0],
 [0, 0, 1, 0, 0],
 [0, 1, 0, 0, 1],
 [0, 1, 0, 0, 1],
 [0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0],
 [0, 1, 0, 1, 0],
 [0, 0, 1, 0, 0],
 [0, 1, 1, 0, 1],
 [0, 1, 0, 0, 1],
 [0, 0, 0, 0, 1],
 [0, 1, 0, 0, 1],
 [1, 0, 0, 0, 0],
 [1, 0, 0, 0, 0],
 [0, 1, 0, 0, 1],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 1, 1, 0, 0],
 [0, 1, 0, 1, 0],
 [0, 1, 0, 0, 0],
 [0, 1, 0, 1, 0],
 [0, 1, 0, 0, 0],
 [0, 1, 0, 1, 0],
 [1, 0, 1, 0, 0],
 [0, 0, 0, 0, 0],
 [1, 1, 0, 0, 0],
 [0, 0, 0, 0, 1],
 [0, 1, 0, 1, 0],
 [1, 0, 0, 0, 0],
 [0, 0, 1, 0, 0],
 [0, 1, 0, 1, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 1],
 [0, 1, 0, 1, 0],
 [0, 0, 0, 0, 0],
 [0, 1, 0, 1, 0],
 [0, 1, 1, 0, 1],
 [0, 0, 0, 0, 0],
 [0, 1, 0, 0, 1],
 [1, 0, 0, 0, 0],
 [0, 1, 0,

In [23]:
df

,id,text,Anger,Fear,Joy,Sadness,Surprise
0,eng_dev_track_a_00001,"My mouth fell open `` No, no, no... I..",NaN,NaN,NaN,NaN,NaN
1,eng_dev_track_a_00002,You can barely make out your daughter's pale f...,NaN,NaN,NaN,NaN,NaN
2,eng_dev_track_a_00003,But after blinking my eyes for a few times lep...,NaN,NaN,NaN,NaN,NaN
3,eng_dev_track_a_00004,Slowly rising to my feet I came to the conclus...,NaN,NaN,NaN,NaN,NaN
4,eng_dev_track_a_00005,I noticed this months after moving in and doin...,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
111,eng_dev_track_a_00112,"""ARcH stop your progression.",NaN,NaN,NaN,NaN,NaN
112,eng_dev_track_a_00113,"This 'star', starts to move across the sky.",NaN,NaN,NaN,NaN,NaN
113,eng_dev_track_a_00114,and my feet hurt.,NaN,NaN,NaN,NaN,NaN
114,eng_dev_track_a_00115,so i cried my eyes out and did the drawing.,NaN,NaN,NaN,NaN,NaN


In [24]:
df_val = pd.DataFrame(pred_labels, columns=['Anger', 'Fear', 'Joy', 'Sadness', 'Surprise'])
df_val.to_csv('my_list.csv', index=False)

In [25]:
df_val.insert(loc=0, column='id', value='text')

In [26]:
df_val

,id,Anger,Fear,Joy,Sadness,Surprise
0,text,0,1,0,0,1
1,text,0,1,0,1,1
2,text,0,0,0,0,1
3,text,0,1,0,1,0
4,text,0,1,0,0,1
...,...,...,...,...,...,...
111,text,0,1,1,0,1
112,text,0,1,0,0,1
113,text,0,1,0,1,0
114,text,0,1,0,1,0


In [27]:
df_val['id'] = [f'eng_dev_track_a_{i+1:05}' for i in range(len(df_val))]
df_val

,id,Anger,Fear,Joy,Sadness,Surprise
0,eng_dev_track_a_00001,0,1,0,0,1
1,eng_dev_track_a_00002,0,1,0,1,1
2,eng_dev_track_a_00003,0,0,0,0,1
3,eng_dev_track_a_00004,0,1,0,1,0
4,eng_dev_track_a_00005,0,1,0,0,1
...,...,...,...,...,...,...
111,eng_dev_track_a_00112,0,1,1,0,1
112,eng_dev_track_a_00113,0,1,0,0,1
113,eng_dev_track_a_00114,0,1,0,1,0
114,eng_dev_track_a_00115,0,1,0,1,0


In [28]:
df_val.to_csv('AUGMENTED_my_pred_after_FT_twitterRoberta.csv', index=False)